In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#%% IMPORTING LIBRARIES 

import os
import glob
import shutil
import json
import keras
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:

work_dir = '../input/cassava-leaf-disease-classification/'
os.listdir(work_dir) 
train_path = '/kaggle/input/cassava-leaf-disease-classification/train_images'

In [ ]:
os.listdir(work_dir) 

In [ ]:
train_path

In [ ]:

data = pd.read_csv(work_dir + 'train.csv')

In [ ]:
data.shape

In [ ]:
data.head(3)

In [ ]:
data['label'].value_counts()

In [ ]:
data['label'].value_counts().plot(kind='bar')

In [ ]:
f = open(work_dir + 'label_num_to_disease_map.json')
real_labels = json.load(f)

In [ ]:
real_labels

In [ ]:
real_labels.items()

In [ ]:
real_labels = {int(k):v for k,v in real_labels.items()}

In [ ]:
real_labels

In [ ]:
data['class_name'] = data.label.map(real_labels)

In [ ]:
data.head(3)

In [ ]:
data['class_name'].value_counts().plot(kind='bar')

In [ ]:
from sklearn.model_selection import train_test_split
train,val = train_test_split(data, test_size = 0.1, random_state = 42, stratify = data['class_name'])

In [ ]:
train.shape

In [ ]:
val.shape

In [ ]:
val['class_name'].value_counts().plot(kind='bar')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMG_SIZE = 224
size = (IMG_SIZE,IMG_SIZE)
n_CLASS = 5

In [ ]:
#resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
from  tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
#model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = resnet_weights_path))

In [ ]:
#tf.keras.applications.resnet.preprocess_input

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:

datagen = ImageDataGenerator(
                    preprocessing_function = tf.keras.applications.vgg16.preprocess_input,
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,
                    fill_mode = 'nearest')

train_set = datagen.flow_from_dataframe(train,
                         directory = train_path,
                         seed=42,
                         x_col = 'image_id',
                         y_col = 'class_name',
                         target_size = size,
                         #color_mode="rgb",
                         class_mode = 'categorical',
                         interpolation = 'nearest',
                         shuffle = True,
                         batch_size = 64)
                         #target_size=(224, 224))

val_set = datagen.flow_from_dataframe(val,
                         directory = train_path,
                         seed=42,
                         x_col = 'image_id',
                         y_col = 'class_name',
                         target_size = size,
                         #color_mode="rgb",
                         class_mode = 'categorical',
                         interpolation = 'nearest',
                         shuffle = True,
                         batch_size = 64)
                         #target_size=(224, 224))


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB3

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras


In [ ]:
import keras
#/kaggle/input/cassava-leaf-disease-classification/train_images/
first_model = keras.models.load_model('/kaggle/input/model-files/Cassava_best_model.h5')

#first_model = 
first_model.summary()

In [ ]:
n_CLASS

In [ ]:
keras.utils.plot_model(first_model)

In [ ]:
EPOCHS = 60
STEP_SIZE_TRAIN = train_set.n//train_set.batch_size
STEP_SIZE_VALID = val_set.n//val_set.batch_size

In [ ]:
train_set.batch_size

In [ ]:
STEP_SIZE_TRAIN

** Main Links that helped
1. https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c

2. https://github.com/1297rohit/VGG16-In-Keras/blob/master/VGG16%20in%20Keras.ipynb

3. https://keras.io/api/preprocessing/image/**

In [ ]:
#%% FITTING THE MODEL

def Model_fit():
    
    #leaf_model = None
    
    main_model = first_model
    
    '''Compiling the model'''
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = False,
                                                   label_smoothing=0.0001,
                                                   name='categorical_crossentropy' )
    
    first_model.compile(optimizer = Adam(learning_rate = 1e-3),
                        loss = loss, #'categorical_crossentropy'
                        metrics = ['categorical_accuracy']) #'acc'
    
    # Stop training when the val_loss has stopped decreasing for 5 epochs.
    es = EarlyStopping(monitor='val_loss', mode='min', patience=5,
                       restore_best_weights=True, verbose=1)
    
    # Save the model with the minimum validation loss
    checkpoint_cb = ModelCheckpoint("Cassava_best_model.h5",
                                    save_best_only=True,
                                    monitor = 'val_loss',
                                    mode='min')
    
    # reduce learning rate
    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                                  factor = 0.3,
                                  patience = 3,
                                  min_lr = 1e-5,
                                  mode = 'min',
                                  verbose = 1)
    
    history = main_model.fit(train_set,
                             validation_data = val_set,
                             epochs= EPOCHS,
                             batch_size = 64,
                             #class_weight = d_class_weights,
                             steps_per_epoch = STEP_SIZE_TRAIN,
                             validation_steps = STEP_SIZE_VALID,
                             callbacks=[es, checkpoint_cb, reduce_lr])
    
    first_model.save('Cassava_model'+'.h5')  
    
    return history

In [ ]:
results = Model_fit()

In [ ]:
#%% PLOTTING RESULTS (Train vs Validation FOLDER 1)

def Train_Val_Plot(acc,val_acc,loss,val_loss):
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize= (15,10))
    fig.suptitle(" MODEL'S METRICS VISUALIZATION ", fontsize=20)

    ax1.plot(range(1, len(acc) + 1), acc)
    ax1.plot(range(1, len(val_acc) + 1), val_acc)
    ax1.set_title('History of Accuracy', fontsize=15)
    ax1.set_xlabel('Epochs', fontsize=15)
    ax1.set_ylabel('Accuracy', fontsize=15)
    ax1.legend(['training', 'validation'])


    ax2.plot(range(1, len(loss) + 1), loss)
    ax2.plot(range(1, len(val_loss) + 1), val_loss)
    ax2.set_title('History of Loss', fontsize=15)
    ax2.set_xlabel('Epochs', fontsize=15)
    ax2.set_ylabel('Loss', fontsize=15)
    ax2.legend(['training', 'validation'])
    plt.show()
    

Train_Val_Plot(results.history['categorical_accuracy'],results.history['val_categorical_accuracy'],
               results.history['loss'],results.history['val_loss'])

In [ ]:
# Evaluating the model

import keras

final_model = keras.models.load_model('Cassava_best_model.h5')

In [ ]:
TEST_DIR = '/kaggle/input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(TEST_DIR)
predictions = []

for image in test_images:
    img = Image.open(TEST_DIR + image)
    img = img.resize(size)
    img = np.expand_dims(img, axis=0)
    predictions.extend(final_model.predict(img).argmax(axis = 1))

In [ ]:
predictions

In [ ]:
sub = pd.DataFrame({'image_id': test_images, 'label': predictions})
display(sub)
sub.to_csv('submission.csv', index = False)